In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e24/sample_submission.csv
/kaggle/input/playground-series-s3e24/train.csv
/kaggle/input/playground-series-s3e24/test.csv


In [3]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
# Import Libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


In [5]:
traindata=pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")
testdata=pd.read_csv("/kaggle/input/playground-series-s3e24/test.csv")


In [6]:
traindata.head(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [7]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [9]:
traindata.isna().sum()

id                     0
age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
dental caries          0
smoking                0
dtype: int64

In [8]:
traindata.drop(['id'],axis=1)

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,155,45,69.0,1.5,2.0,1,1,127,80,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,50,155,75,82.0,1.0,1.0,1,1,120,80,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,40,160,50,66.0,1.5,1.0,1,1,114,70,...,87,93,10.9,1,0.6,15,9,12,0,0
159254,50,165,75,92.0,1.2,1.0,1,1,121,90,...,55,80,14.4,1,1.1,22,17,37,0,1


In [10]:
X=traindata.drop(['smoking'],axis=1,inplace=False)
y=traindata['smoking']

In [12]:
y.shape

(159256,)

In [13]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:

RandomForestClassifierModel = RandomForestClassifier(criterion = 'gini',n_estimators=100,max_depth=2,random_state=33) #criterion can be also : entropy
RandomForestClassifierModel.fit(x_train, y_train)
#Calculating Prediction
RFC_y_pred = RandomForestClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, RFC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, RFC_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, RFC_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, RFC_y_pred)
print(f"F1-Score: {f1}")

Accuracy Score: 0.7307861358784378
Precision: 0.6777303312629399
Recall: 0.7445447437628829
F1-Score: 0.7095681625740897


In [16]:
DecisionTreeClassifierModel = DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=33) #criterion can be entropy
DecisionTreeClassifierModel.fit(x_train, y_train)

#Calculating Prediction
DTC_y_pred = DecisionTreeClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, DTC_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, DTC_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, DTC_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, DTC_y_pred)
print(f"F1-Score: {f1}")

Accuracy Score: 0.7257315082255431
Precision: 0.6919035624325297
Recall: 0.6833463643471462
F1-Score: 0.6875983407237878


In [17]:

KNNClassifierModel = KNeighborsClassifier(n_neighbors= 5,weights ='uniform', # it can be distance
                                          algorithm='auto') # it can be ball_tree, kd_tree,brute
KNNClassifierModel.fit(x_train, y_train)

#Calculating Prediction
KNN_y_pred = KNNClassifierModel.predict(x_test)
# Calculate accuracy
val_accuracy = accuracy_score(y_test, KNN_y_pred)
print(f"Accuracy Score: {val_accuracy}")
# Calculate precision
precision = precision_score(y_test, KNN_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, KNN_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, KNN_y_pred)
print(f"F1-Score: {f1}")

Accuracy Score: 0.6566620620369208
Precision: 0.6150400235000367
Recall: 0.5952804037245006
F1-Score: 0.6049989164198512


In [18]:
# Initializing CatBoostClassifier
CatBoostClassifierModel = CatBoostClassifier(iterations=100,
                                             depth=2,
                                             learning_rate=0.1,
                                             random_seed=33,
                                             logging_level='Silent')

# Fitting the model
CatBoostClassifierModel.fit(x_train, y_train)

# Calculating Prediction
CatBoost_y_pred = CatBoostClassifierModel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, CatBoost_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, CatBoost_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, CatBoost_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, CatBoost_y_pred)
print(f"F1-Score: {f1}")

Accuracy Score: 0.7628720331533342
Precision: 0.7026119402985075
Recall: 0.8030421494064965
F1-Score: 0.74947759461342


In [23]:
XGBoostClassifierModel = XGBClassifier(n_estimators=100,
                                           max_depth=2,
                                           learning_rate=0.1,
                                           random_state=33,
                                           verbosity=0)

# Fitting the model
XGBoostClassifierModel.fit(x_train, y_train)

# Calculating Prediction
XGBoost_y_pred = XGBoostClassifierModel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, XGBoost_y_pred)
print(f"Accuracy Score: {val_accuracy:.2f}")

# Calculate precision
precision = precision_score(y_test, XGBoost_y_pred)
print(f"Precision: {precision:.2f}")

# Calculate recall
recall = recall_score(y_test, XGBoost_y_pred)
print(f"Recall: {recall:.2f}")

# Calculate F1-score
f1 = f1_score(y_test, XGBoost_y_pred)
print(f"F1-Score: {f1:.2f}")

Accuracy Score: 0.77
Precision: 0.70
Recall: 0.81
F1-Score: 0.75


In [19]:
testdata.head(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,159256,40,165,70,84.0,1.2,1.2,1,1,130,...,186,49,115,14.2,1,0.9,19,25,32,0
1,159257,80,160,60,93.0,1.0,1.0,2,2,144,...,158,35,104,13.0,1,1.1,20,12,24,0
2,159258,60,170,70,86.5,0.6,0.7,1,1,117,...,173,39,88,15.4,1,1.4,38,60,36,0
3,159259,40,160,50,67.0,0.3,0.4,1,1,116,...,47,75,128,14.5,1,0.6,25,18,10,1
4,159260,40,170,75,89.4,1.0,0.9,1,1,132,...,100,39,123,16.5,1,1.0,30,39,27,1


In [20]:
test_predictions = CatBoostClassifierModel.predict(testdata.drop(['id'], axis=1))

# Now create the submission DataFrame
submission = pd.DataFrame({'id': testdata['id'], 'smoking': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created successfully!")


Submission file created successfully!
